# Deploy IoT Edge Modules

In this section, we will deploy LVA & Inference Server modules (IoT Edge Runtime specific Docker container images) to an IoT Edge device.

## Key Concepts
In the first part of this section, we will write a manifest file in Json format and specify the details of the modules (IoT Edge Runtime specific Docker container images) that will be deployed into the Edge device. We will use the [deployment.lva_common.template.json](deployment.lva_common.template.json) manifest file as a template for module deployment. Familiarize yourself with the contents of this manifest file template. The manifest is organized similarly to the following image:  

<img src="../../../../images/_deployment_modules_0.png" width=300px/>  

As shown by the enumeration in the image above, branch 2 of the Json file defines the system modules to be deployed onto the Edge device. These two system modules (edgeAgent, edgeHub) are necessary modules in each IoT Edge device. In branch 3 of the Json file, we define the custom modules that we want to deploy to the edge device.  
The three custom modules in branch 3 of the Json file are as follows: 
* lvaEdge - node where we define the lvaEdge-specific details, such as from which Container Registry address it should be downloaded. (You may use other reference names, but the name must be consistent in other locations within the manifest file or in any other related code.)
* isServer - the reference name we gave for our inference server module that we developed and uploaded into ACR.
* rtspsim - the reference name given to another container image in ACR. This module simulates an IP camera by playing a user-defined video file as if it were being streamed from an IP camera. This module is optional and placed here to aid users who don't have an IP camera but still want to test LVA.

For more details about IoT Edge modules, refer to [Azure IoT Edge documentation](https://docs.microsoft.com/en-us/azure/iot-edge/about-iot-edge).

If you check the contents of the template manifest file, you will see placeholders that start with the character $. 

| **Placeholder** | **Description** | 
| --- | --- | 
| CONTAINER_REGISTRY_USERNAME_myacr | The username that Azure Container Registry (ACR) will use to authenticate with the registry.<br>ACR is a registry of Docker and Open Container Initiative (OCI) images.| 
| CONTAINER_REGISTRY_PASSWORD_myacr | The password that Azure Container Registry (ACR) will use to authenticate with the registry.<br>When run, a Active Directory token is created to authenticate your session with the container registry.| 
| CONTAINER_IMAGE_NAME | The name of the container image that will be used for deployment.<br>Once uploaded to ACR, it can be called through CONTAINER_REGISTRY_USERNAME_myacr.azurecr.io/CONTAINER_IMAGE_NAME. |
| SUBSCRIPTION_ID | The ID associated with your Azure subscription, typically containing dashes within the string.<br>A subscription represents a grouping of Azure resources. An invoice is generated at the subscription scope. | 
| RESOURCE_GROUP | The name of the resource group that holds your resources.<br>A resource group is a container that holds related resources for an Azure solution. | 
| AMS_ACCOUNT | The name of your Azure Media Services (AMS) resource.<br>AMS is a cloud-based media workflow platform to index, package, protect, and stream video. |
| AAD_TENANT_ID | The Azure Active Directory (AAD) tenant ID, which is used as a reference to a directory.<br>An AAD directory object holds security-related objects such as user accounts, applications, and groups.| 
| AAD_SERVICE_PRINCIPAL_ID | The ID associated with the AAD's service principal.<br>A service principal is a security identity used by user-created apps, services, and tools to access Azure resources. | 
| AAD_SERVICE_PRINCIPAL_SECRET | A secret string associated with the AAD's service principal.<br>The AAD service principal secret adds a layer of security at the service principal scope. | 
| INPUT_VIDEO_FOLDER_ON_DEVICE | The path pointing to the folder in which the desired input video is stored.<br>This path must be specified in order for LVA to find the correct videos to process. | 
| OUTPUT_VIDEO_FOLDER_ON_DEVICE | The path pointing to the folder in which the processed video output will be stored.<br>After processing is complete, the processed video will be sent and stored in this folder. | 

In the [previous section](07_upload_container_image_to_acr.ipynb), we pushed our inference server container image to the cloud using Azure Container Registry (ACR). In doing so, ACR created a .env file containing the credentials to your container registry. These credentials are shared with your IoT Edge device so that it has access to pull the container image.

We will use the credentials found in the .env file to create an IoT Edge device module deployment manifest file in Json format. The credentials found in the .env file will replace the placeholders we set in our template [deployment.lva_common.template.json](deployment.lva_common.template.json) manifest file.

**How does this work?** The deployment manifest file is a file that defines exactly which modules you want deployed on a device, how they should be configured, and how they can communicate with each other and the cloud. When you transform the template into a true deployment manifest, the placeholders in the table above are replaced with values taken from other solution files. Credentials placed in this file are then shared with the IoT Edge runtime to pull your container images onto the IoT Edge device. This configuration is for security purposes, as the .env file is git ignored, but the deployment template is not. You should double check by opening the .env file and checking if your credentials are included. If not, add them manually by locating the appropriate services on Azure and save the .env file.

## Setting Up the Deployment Manifest File

In [1]:
from env_variables import *
set_key(envPath, "INPUT_VIDEO_FOLDER_ON_DEVICE", "/lvafolders/input")
set_key(envPath, "OUTPUT_VIDEO_FOLDER_ON_DEVICE", "/lvafolders/output")

(True, 'OUTPUT_VIDEO_FOLDER_ON_DEVICE', '/lvafolders/output')

## Setting Up Local Folders on the IoT Edge Device
In this step, we will set up local folders on our IoT Edge device. In some deployment manifests, it may not be necessary to create these additional folders. However, we will create these folders in this section as a placeholder for later deployments.

First, connect to the Iot Edge device in a terminal window and create the two folders below on the IoT Edge device.

Using terminal window, create two folders in any local location in your Edge device. We prefer the following folder paths:

```
    /lvafolders/input  
    /lvafolders/output  
```

You can give these folders any name and create them under any desired root folder. However, the full path must be same in the other parts of this section. Both folders must have read and write access for all users. One way to create these folders is by using the commands below, which create the folders and set their access permissions.

Run the commands below **on the IoT Edge device** through a terminal window.

```
sudo mkdir -p /lvafolders/input
sudo mkdir /lvafolders/output
```

Next, set the access permissions for these folders.
```
sudo chmod 777 -R /lvafolders
```

Afterwards, in your IoT Edge device's terminal window, run the following command to download the video sample named `lots_284.mkv` into your /lvafolders/input folder. This video sample is a minute-long clip of a parking lot.

```
curl https://lvamedia.blob.core.windows.net/public/lots_284.mkv --output /lvafolders/input/lots_284.mkv
```

In order to use the OpenVINO lacense plate detection model, copy the sample files from the the `/live-video-analytics/media` folder to the `/lvafolders/input` folder.

Finally, set the access permissions again for safe measure.  
```
sudo chmod 777 -R /lvafolders
```


## Reset the IoT Edge Device and Deploy the LVA & Inference Server Modules

Next, we will reset the IoT Edge device by uploading a deployment manifest that consists of only two system modules: IoTHub and IoTEdge.

1. Right click on the "deployment.reset.json" file  


<img src="../../../../images/_deployment_reset.png" width=500px/>  


2. Click on "Create Deployment for Single Device"  
    a. If it is the first time using the IoT Hub service that was created in the previous sections, VSCode will ask you to enter the "IoT Hub Connection String". You can use the value of the "IOT_HUB_CONN_STRING" key, which can be located in the .env file.  
    b. The "Create Deployment for Single Device" command will open a window on top edge of the VSCode and will ask for the IoT Edge device name that you want to use make this deployment. Since you entered the IoT Hub name in step (a), it will automatically find names of applicable devices and list them for you. For this section, we only have one Edge device, which we named "iotDeviceId" from the [second section](02_setup_environment.ipynb) when we defined the variables. Select this device in the drop-down list.  

    
<img src="../../../../images/_select_edge_device.png" width=500px/>  


3. Locate the deployment manifest that you created in previous sections. Note that the name of this file will not be "deployment.lva_common.template.json" because you overrode some paramters and thus created a new file. Right click on the deployment manifest file. In the pop-up menu, click on "Generate IoT Edge Deployment Manifest". In this step, VSCode will auto-read the contents of the .env file and use the values of the variables to write over some placeholder variables inside the file. After these replacements, a new version of the file with the name "deployment.lvaedge_isedge_rtspsim.amd64.json" will be placed under the newly created "config" folder in the same working directory.  


<img src="../../../../images/_deployment_modules_1.png" width=600px/>  


4. Right click on the file "/config/deployment.lvaedge_isedge_rtspsim.amd64.json". In the pop-up menu, click on "Create Deployment for Single Device".  


<img src="../../../../images/_deployment_modules_2.png" width=600px/>  


5. Like in step 2(b), VSCode will ask the device name to where the deployment will be made. Here again, select the device name "iotDeviceId" (or whichever name was used in the [second section](02_setup_environment.ipynb) when we defined the variables).

6. Depending on your Internet speed, the modules will be pulled from the cloud and deployed into your Edge device; this process can range from seconds to minutes. In VSCode, refresh the "AZURE IOT HUB" panel. You can see the modules that are uploaded on to your Edge device once the refresh is complete.  

<img src="../../../../images/_iotedgedevice_view.png" width=400px/>  


As in the screen shot above, you should see a status of "running" next to each module name (see region 3 in the above screenshot).  

Alternatively, you can run the following command in a terminal window on the Iot Edge device:  

```
sudo iotedge list
```

The result of the command above will look something similar to the snippet below, with all module statuses saying "running".

```
mksa@mknuc02:~/Desktop/aiext$ sudo iotedge list
[sudo] password for mksa: 
NAME             STATUS           DESCRIPTION      CONFIG
isEdge           running          Up 6 minutes     mkregistry:55000/isedge:latest
edgeAgent        running          Up 7 minutes     mcr.microsoft.com/azureiotedge-agent:1.0
edgeHub          running          Up 6 minutes     mcr.microsoft.com/azureiotedge-hub:1.0
lvaEdge          running          Up 6 minutes     lvacontainers.azurecr.io/lvaedge:1
rtspsim          running          Up 6 minutes     mcr.microsoft.com/lva-utilities/rtspsim-live555:1.2
```

If all the modules are running, you have succesfully deployed the modules into the Iot Edge device.  

## Next Steps
If all the code cells above have successfully finished running, return to the Readme page to continue.   